In [1]:
!pip install linearmodels


     |████████████████████████████████| 1.2MB 2.9MB/s 
  Created wheel for linearmodels: filename=linearmodels-4.13-py2.py3-none-any.whl size=802536 sha256=53374b58cf235058574d4ef5458ac7df57fbe6c95c0b846bae5b73424e068a19
  Stored in directory: /root/.cache/pip/wheels/92/33/3a/5efd7cb50597ff3dc29256ba54e36e2a2c820fd1d1069ea967
Successfully built linearmodels


In [0]:
import pandas as pd
from linearmodels import IV2SLS, IV3SLS, IVSystemGMM

In [3]:
file="https://github.com/VitorKamada/ECO7110/raw/master/Data/mroz.dta"
df = pd.read_stata(file)
df.head()

,inlf,hours,kidslt6,kidsge6,age,educ,wage,repwage,hushrs,husage,huseduc,huswage,faminc,mtr,motheduc,fatheduc,unem,city,exper,nwifeinc,lwage,expersq
0,1,1610,1,0,32,12,3.3540,2.65,2708,34,12,4.0288,16310.0,0.7215,12,7,5.0,0,14,10.910060,1.210154,196
1,1,1656,0,2,30,12,1.3889,2.65,2310,30,9,8.4416,21800.0,0.6615,7,7,11.0,1,5,19.499981,0.328512,25
2,1,1980,1,3,35,12,4.5455,4.04,3072,40,12,3.5807,21040.0,0.6915,12,7,5.0,0,15,12.039910,1.514138,225
3,1,456,0,3,34,12,1.0965,3.25,1920,53,10,3.5417,7300.0,0.7815,7,7,5.0,0,6,6.799996,0.092123,36
4,1,1568,1,2,31,14,4.5918,3.60,2000,32,12,10.0000,27300.0,0.6215,12,14,9.5,1,7,20.100058,1.524272,49


a. Estimate equations (9.28) and (9.29) jointly by 3SLS, and compare the 3SLS estimates with the 2SLS estimates for equations (9.28) and (9.29)

In [4]:
Eq1 ='hours ~ educ + age + kidslt6 + kidsge6 + nwifeinc + [lwage ~ exper + expersq]'

Hours = IV2SLS.from_formula(Eq1, df).fit()

print(Hours)


                          IV-2SLS Estimation Summary                          
Dep. Variable:                  hours   R-squared:                      0.0861
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0731
No. Observations:                 428   F-statistic:                    425.64
Date:                Mon, Oct 21 2019   P-value (F-stat)                0.0000
Time:                        21:29:46   Distribution:                  chi2(6)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
educ          -121.29     66.801    -1.8157     0.0694     -252.22      9.6361
age            20.077     6.3043     3.1846     0.00

/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [5]:
Eq2 = 'lwage ~ educ + exper + expersq [hours ~ age + kidslt6 + kidsge6 + nwifeinc]'
Lwage = IV2SLS.from_formula(Eq2, df).fit()
print(Lwage)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.7533
Estimator:                    IV-2SLS   Adj. R-squared:                 0.7510
No. Observations:                 428   F-statistic:                    1473.7
Date:                Mon, Oct 21 2019   P-value (F-stat)                0.0000
Time:                        21:29:53   Distribution:                  chi2(4)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
educ           0.0891     0.0182     4.8935     0.0000      0.0534      0.1248
exper          0.0553     0.0339     1.6328     0.10

/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [6]:
Eq = dict(hours= Eq1, lwage = Eq2)
IV3SLS = IV3SLS.from_formula(Eq, df).fit (method='gls')
print(IV3SLS)


                        System GLS Estimation Summary                         
Estimator:                        GLS   Overall R-squared:             -0.0476
No. Equations.:                     2   Cov. Estimator:                 robust
No. Observations:                 428   Num. Constraints:                 None
Date:                Mon, Oct 21 2019                                         
Time:                        21:30:06                                         
                                                                              
                                                                              
                  Equation: hours, Dependent Variable: hours                  
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
educ          -120.97     22.700    -5.3288     0.0000     -165.46     -76.475
age            13.868     4.5130     3.0730     0.00

/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


The 3SLS method produced results with lower standard errors for equation 9.28 but higher standard errors for equation 9.29.  

b. Now allow educ to be endogenous in equation (9.29). but assume it is exogenous in equation (9.28). Estimate a three equation system using different instruments for different equations, where motheeduc, fatheduc, and huseduc are assumed exogenous in equations (9,28) and (9.29)

In [8]:

Eq1 ='hours ~ educ + age +  kidslt6 + kidsge6 + nwifeinc + motheduc + fatheduc + huseduc + [lwage  ~ exper + expersq]'

Eq2 = 'lwage ~ [hours + educ ~ age + kidslt6 + kidsge6 + nwifeinc]+ exper + expersq + motheduc + fatheduc + huseduc'

Eq3 = 'educ ~ age + kidslt6 + kidsge6 + nwifeinc + exper + expersq + motheduc + fatheduc + huseduc'


Eq = dict(hours= Eq1,lwage=Eq2, educ=Eq3)
system_gmm = IVSystemGMM.from_formula(Eq, df, weight_type='robust')
system_gmm_res = system_gmm.fit(cov_type='robust', iter_limit=100)
print('Number of iterations: ' + str(system_gmm_res.iterations))
print(system_gmm_res)





Number of iterations: 20
                 System Iterative System GMM Estimation Summary                 
Estimator:         Iterative System GMM   Overall R-squared:             -0.0808
No. Equations.:                       3   Cov. Estimator:                 robust
No. Observations:                   428   Num. Constraints:                 None
Date:                  Mon, Oct 21 2019                                         
Time:                          21:30:18                                         
                                                                                
                                                                                
                  Equation: hours, Dependent Variable: hours                  
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
educ          -214.16     96.410    -2.2214     0.0263     -403.12     -25.201
age        

/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
